In [3]:
import pgmpy 
from pgmpy.models import BayesianModel
from pgmpy.factors.discrete import TabularCPD
import pandas as pd
import numpy as np

In [18]:
url = 'https://raw.githubusercontent.com/wshenyc/MLForCities/main/Data/housing_data.csv'
df = pd.read_csv(url)
#df.set_index("bbl", inplace = True)
df.set_index("cd", inplace = True)
df = df.replace('?',np.nan)
df.dropna(inplace=True, how='any')
df.head()

,bbl,county,tract10,res_units,other_units,year_built,year_reno,buildings,floors,building_class,...,viol_bbl_oth_2018,viol_bbl_oth_2019,viol_bbl_oth_2020,viol_bbl_oth_2021,viol_bbl_ser_2017,viol_bbl_ser_2018,viol_bbl_ser_2019,viol_bbl_ser_2020,viol_bbl_ser_2021,outcome
cd,,,,,,,,,,,,,,,,,,,,,
316.0,3035090064,47,47090000.0,39,0,1930.0,1982.0,1,4.0,C1,...,2.000000,0.153846,0.000000,5.051282,2.000000,0.025641,1.051282,0.000000,0.076923,True
307.0,3007810074,47,47002000.0,8,0,1920.0,2006.0,1,4.0,C1,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,False
112.0,1021430009,61,61026300.0,49,0,1925.0,1988.0,1,6.0,D1,...,6.469388,5.326531,0.204082,3.102041,3.102041,0.061224,0.306122,0.040816,0.000000,False
108.0,1015270006,61,61013800.0,14,0,1910.0,1988.0,1,6.0,C5,...,4.000000,2.357143,0.214286,0.142857,0.000000,0.000000,0.000000,0.214286,0.000000,False
107.0,1012000052,61,61017300.0,7,1,1892.0,1892.0,1,4.0,C7,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304.0,3031540001,47,47042500.0,61,0,2002.0,2002.0,1,7.0,D1,...,0.344262,0.262295,1.081967,1.049180,0.000000,0.000000,1.114754,0.032787,1.098361,True
111.0,1017490066,61,61019800.0,36,4,1900.0,1997.0,4,5.0,C7,...,4.138889,1.138889,0.277778,2.138889,2.027778,1.000000,0.000000,0.083333,0.027778,True
101.0,1000167520,61,61031703.0,280,2,2008.0,2008.0,1,32.0,RM,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,False


In [31]:
from sklearn.cluster import KMeans

df2 = df
df2.iloc[:,0:] = df2.iloc[:,0:].apply(pd.to_numeric,errors='coerce',downcast='integer')
#df2 = df2.dropna(how='any')
df2 = df2.dropna(axis="columns")


KM = KMeans(n_clusters=3,random_state=522)
scor=KM.fit_predict(df2)

array([1, 1, 0, ..., 0, 2, 2], dtype=int32)

In [38]:
res_km=pd.DataFrame(KM.transform(df2))
res_km.index = df2.index
res_km.columns=list(range(3))
res_km["nearest_dist"] = res_km.min(axis=1)
res_km['cluster'] = res_km.idxmin(axis=1)
test = res_km.sort_values(by= ["nearest_dist"], ascending=False).head(5)
test

,0,1,2,nearest_dist,cluster
cd,,,,,
503.0,4.054593e+09,1.871315e+09,3.041048e+09,1.871315e+09,1
503.0,4.053033e+09,1.869755e+09,3.039489e+09,1.869755e+09,1
503.0,4.050793e+09,1.867515e+09,3.037249e+09,1.867515e+09,1
502.0,4.027503e+09,1.844229e+09,3.013967e+09,1.844229e+09,1
503.0,4.027173e+09,1.843899e+09,3.013638e+09,1.843899e+09,1


In [ ]:
from collections import defaultdict
def temp_predict(model,data):
    missing_variables = list(set(model.nodes()) - set(data.columns))
    pred_values = defaultdict(list)
    model_inference = VariableElimination(model)
    for index, data_point in data.iterrows():
        states_dict = model_inference.map_query(variables=missing_variables, evidence=data_point.astype(int).to_dict())
        for k, v in states_dict.items():
            pred_values[k].append(v)
    return pd.DataFrame(pred_values, index=data.index)

train_x = train.loc[:,"tip":"pass"]
#predicted_train = best_model.predict(train_x) should work, but is buggy in the current version
predicted_train = temp_predict(best_model, train_x)
print "In sample:",(train.loc[:,'manhattan']==predicted_train['manhattan']).mean()

test_x = test.loc[:,"tip":"pass"]
#predicted_test = best_model.predict(test_x) should work, but is buggy in the current version
predicted_test = temp_predict(best_model, test_x)
print "Out of sample:",(test.loc[:,'manhattan']==predicted_test['manhattan']).mean()